In [ ]:
!pip install transformers
!pip install torch
!pip install torchvision
!pip install pillow


In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
from io import BytesIO

In [ ]:
images=["https://th.bing.com/th?id=OIP.M-bg1y9cFEEt8RpkafzOTwHaFj&w=288&h=216&c=8&rs=1&qlt=90&o=6&dpr=1.5&pid=3.1&rm=2",
        "https://thumbs.dreamstime.com/z/kwame-nkrumah-memorial-park-monument-knmp-national-accra-ghana-named-osagyefo-dr-48650280.jpg",
        "https://www.zdnet.com/a/img/resize/49b1ec66e176430592d81ce6aa37aa499bfb4abb/2023/09/13/750409b0-0d4e-4bc9-90f2-16565bae042c/iphone-15-pro-max-white-1.jpg?auto=webp&precrop=2048,1151,x0,y12&width=1280",
        "https://th.bing.com/th/id/OIP.R02cmN97jMvlUAboM9BPJQHaE5?rs=1&pid=ImgDetMain",
        "http://wondrlust.com/wp-content/uploads/2015/10/happiness-960x540.jpg",
        "https://www.tastefullyoffensive.com/wp-content/uploads/2021/01/1bf4f15dbcb6f8923a59f60eae-confusing-pictures.jpg",
        "https://www.anopticalillusion.com/wp-content/uploads/2020/10/duck-rabbit-optical-illusion.png",
        "https://img.freepik.com/free-photo/world-cuisine-with-delicious-food_23-2151890014.jpg?t=st=1733420813~exp=1733424413~hmac=9a6c3be25c7aa82fa6bbf5c6eccc0f520234c50294940cd266653a7d1c185c8b&w=1060",
        "https://th.bing.com/th/id/OIP.jSfJczEBk-t89yYiEaJ7QwHaFC?w=227&h=180&c=7&r=0&o=5&dpr=2&pid=1.7",
        "https://www.soccerbible.com/media/117952/balls-6-min.jpg"

        ]

In [ ]:
len(images)

10

In [ ]:



# Function to generate image description
def generate_image_description(image_url):
    # Load the BLIP model and processor
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    # Download the image
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).convert('RGB')

    # Display the image
    display(img)

    # Prepare the image for the model
    inputs = processor(images=img, return_tensors="pt")

    # Generate description
    out = model.generate(**inputs)
    description = processor.decode(out[0], skip_special_tokens=True)

    return description

# Example usage
# image_url = 'https://example.com/path_to_your_image.jpg'
#description = generate_image_description(image_url)
#print("Generated Description:")
#print(description)


In [ ]:
for i in images:
  print(generate_image_description(i))

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM


In [ ]:
from transformers import logging
logging.set_verbosity_error()



# Function to generate image description
def generate_image_description_git(image_url):
    # Load the BLIP model and processor
    processor = AutoProcessor.from_pretrained("microsoft/git-base-textcaps")
    model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-textcaps")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Download the image
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).convert('RGB')

    # Display the image
    display(img)

    # Prepare the image for the model
    inputs = processor(images=img, return_tensors="pt")
   # pixel_values = processor(images=img, return_tensors="pt").pixel_values

    # Generate description
    pixel_values = inputs.pixel_values.to(device)

    generated_ids = model.generate(pixel_values=pixel_values, max_length=50)

    return("Generated caption:", processor.batch_decode(generated_ids, skip_special_tokens=True))


# Example usage
# image_url = 'https://example.com/path_to_your_image.jpg'
#description = generate_image_description(image_url)
#print("Generated Description:")
#print(description)


In [ ]:
for i in images:
  print(generate_image_description_git(i))

In [ ]:
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer


# Function to generate image description
def generate_image_description_gpt2(image_url):
    # Load model components
    model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
    tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    max_length = 16
    num_beams = 4
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

    # Download the image
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content)).convert('RGB')

    # Display the image
    display(img)

    # Prepare the image for the model
    pixel_values = feature_extractor(images=img, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

 # Generate predictions
    output_ids = model.generate(pixel_values, **gen_kwargs)

    # Decode the prediction and return the caption
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    return caption


In [ ]:
for i in images:
  print(generate_image_description_gpt2(i))

Picture 1(A man and woman walking down the street| OK
A man and a woman are standing in front of a woman.| GOOD
a man and a woman standing next to each other| OK
)  
Picture 2(Statue of buddha in the temple of the buddha| FALSE
A statue of a man pointing to the right with a blue background.| GOOD
A statue of a man sitting on top of a stone wall|OK
)
Picture 3(The iphone 11 is seen in this photo taken from the back of the phone 11| GOOD
A phone with the back of a phone that has a lens on it| GOOD
a white remote control sitting on top of a white table|FALSE

)
Picture 4(A diagram of the plant's life|FALSE
A picture of photosynthesis with photosynthesis| GOOD
a vase filled with flowers on top of a table| BAD
)
Picture 5(A woman in a yellow shirt holding an umbrella|OK
A girl with her arms outstretched with her arms outstretched in a field with a multicolored umbrella.|GOOD
a woman is flying a colorful kite in the sky| FALSE



)
Picture 6(A man kneeling down to pet a horse| GOOD
A man kneeling down next to a horse in a field|GOOD
a brown horse standing on top of a lush green hillside| GOOD

)
Picture 7(A black and white drawing of a duck|OK
A black and white drawing of a bird with a black and white background.|GOOD
a statue of a bird on a wall|FALSE

)
Picture 8(A pizza with pepperoni and cheese|GOOD
A pizza with pepperoni and cheese on it sits on a table.|BETTER
a pizza sitting on top of a metal pan|OK

)
Picture 9(A toy car sitting on the ground|GOOD
'A red car with a fire extinguisher on the side.|OK
a red and white double decker bus parked on a sidewalk|BAD

)
Picture 10(A soccer ball on the grass|GOOD
A soccer ball with a picture of a horse on it|BETTER
A soccer ball on a field with grass|GOOD
)

BLIP
Microsoft GIT
VIT-GPT2